# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from config import google_api_key
# you are here *
os.getcwd()

'C:\\Users\\Alec\\Documents\\Programming Resources\\Boot Camp\\06_API\\06_API_Homework\\VacationPy'

In [2]:
# Load the csv exported in Part I to a DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_data_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mahébourg,-20.4081,57.7000,74.25,73,68,13.31,MU,1634594791
1,Punta Arenas,-53.1500,-70.9167,50.11,53,40,29.93,CL,1634594791
2,Rikitea,-23.1203,-134.9692,75.47,72,22,20.80,PF,1634594525
3,Cape Town,-33.9258,18.4232,60.12,58,0,3.00,ZA,1634594570
4,Bluff,-46.6000,168.3333,49.21,48,99,24.90,NZ,1634594791


### Humidity Heatmap

In [3]:
#Configure gmaps. double check parameters
gmaps.configure(api_key=google_api_key)

In [6]:
# Use the Lat and Lng as locations 
# double [[]] for multi column action
locations = city_data_df[['Latitude','Longitude']]

# Humidity as the weight.
weights = city_data_df['Humidity']
 
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.
# https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#heatmaps

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations,weights, dissipating=True,)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# To avoid re-drawing the whole map every time you tweak these settings, 
# set them in another notebook cell:

heatmap_layer.max_intensity = 400
heatmap_layer.point_radius = 5

### Create new DataFrame fitting weather criteria

In [ ]:
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values
# look into line structure for pythonic conditonal statements
# 2.1.5
#https://docs.python.org/3/reference/lexical_analysis.html#line-structure

narrow_city_df = city_data_df.loc[(city_data_df["Max Temp"].between(70,80) \
                                   & (city_data_df["Wind Speed"] < 10) \
                                   & (city_data_df["Cloudiness"] == 0))].dropna()
narrow_city_df

# Hotel Map

In [ ]:
# Store into variable named `hotel_df`.
# Add a "Hotel Name" column to the DataFrame.

# keep only the columns we need
# df=old_df[['columns','you','want']] doesnt work | use .copy() to avoid "setting with copy warning"
hotel_df = narrow_city_df[['City', 'Country', 'Latitude', 'Longitude']].copy()
hotel_df['Hotel Name'] = ''
hotel_df

In [ ]:
# https://developers.google.com/maps/documentation/places/web-service/search-nearby
# https://developers.google.com/maps/documentation/places/web-service/supported_types
# Set parameters to search for hotels with 5000 meters.

params = {
    'radius': 5000,
    'key': google_api_key,
    'type': 'lodging'
}

# Hit the Google Places API for each city's coordinates.
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iterrows.html

# loop through dataframe and set variable = values we need
# look up try/except to make error resistant code like in example chapter 6.3
for index,row in hotel_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # change the params dict per loop by creating a key:value inside loop
    params['location'] = f'{latitude},{longitude}'
    
    # start API Call now that we have parameters
    
    # JSON view this call to make pathway for hotel_name
    # print(json.dumps(response, indent=4, sort_keys=True))
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    hotel_name = response
    
# Store the first Hotel result into the DataFrame.
    try:
        # set index as row value in loc
        hotel_df.loc[index, 'Hotel Name'] = hotel_name['results'][0]['name']

    except(IndexError, KeyError):
        print("Nothing Found... Moving on.")

        
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig